## 실전 워크숍

### 도형 알아맞히기

In [3]:
import cv2
import numpy as np

img = cv2.imread('./img/5shapes.jpg')
img2 = img.copy()
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
    vertices = len(approx)
    print('vertices:', vertices)
    
    mmt = cv2.moments(contour)
    cx, cy = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])
    
    name = 'Unknown'
    if vertices == 3:
        name = 'Triangle'
        color = (0,255,0)
    elif vertices == 4:
        x,y,w,h = cv2.boundingRect(contour)
        if abs(w-h)<=3:
            name = 'Square'
            color = (0,125,255)
        else:
            name = 'Rectangle'
            color = (0,0,255)
    elif vertices == 10:
        name = 'Star'
        color = (255,255,0)
    elif vertices >= 15:
        name = 'Circle'
        color = (0,255,255)
    
    cv2.drawContours(img2, [contour], -1, color, -1)
    cv2.putText(img2, name, (cx-50, cy), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (100,100,100), 1)
    
cv2.imshow('Input Shapes', img)
cv2.imshow('Recognizing Shapes', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

vertices: 10
vertices: 16
vertices: 3
vertices: 4
vertices: 4


### 문서 스캐너

In [5]:
import cv2
import numpy as np

win_name = 'scan'
img = cv2.imread('./img/paper.jpg')
cv2.imshow('original', img)
cv2.waitKey(0)
draw = img.copy()

# 그레이 스케일 변환 및 캐니 엣지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0)  # 가우니시안 블러로 노이즈 제거
edged = cv2.Canny(gray, 75, 200)   # 캐니 엣지로 경꼐 검출
cv2.imshow(win_name, edged)
cv2.waitKey(0)

# 컨투어 찾기
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 모든 컨투어 그리기
cv2.drawContours(draw, cnts, -1, (0,255,0))
cv2.imshow(win_name, draw)
cv2.waitKey(0)

# 컨투어들 중에 영역 크기순으로 정렬
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    peri = cv2.arcLength(c, True)
    vertices = cv2.approxPolyDP(c, 0.02*peri, True)
    if len(vertices) == 4:
        break
pts = vertices.reshape(4,2)
for x,y in pts:
    cv2.circle(draw, (x,y), 10, (0,255,0), -1)
cv2.imshow(win_name, draw)
cv2.waitKey(0)
merged = np.hstack((img, draw))

sm = pts.sum(axis=1)
diff = np.diff(pts, axis=1)
            
topLeft = pts[np.argmin(sm)]    
bottomRight = pts[np.argmax(sm)]
topRight = pts[np.argmin(diff)]  
bottomLeft = pts[np.argmax(diff)]
            
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
            

w1 = abs(bottomRight[0] - bottomLeft[0]) 
w2 = abs(topRight[0] - topLeft[0])       
h1 = abs(topRight[1] - bottomRight[1])   
h2 = abs(topLeft[1] - bottomLeft[1])     
width = int(max([w1, w2]))       
height = int(max([h1, h2]))   
            
pts2 = np.float32([[0,0], [width-1,0], [width-1,height-1], [0,height-1]])

mtrx = cv2.getPerspectiveTransform(pts1, pts2)

result = cv2.warpPerspective(img, mtrx, (width, height))
          
cv2.imshow(win_name, result)
cv2.waitKey(0)
cv2.destroyAllWindows()
            

### 동전 개수 세기

In [17]:
import cv2
import numpy as np

img = cv2.imread('./img/coins_connected.jpg')
rows, cols = img.shape[:2]
cv2.imshow('original', img)

mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)

gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0)

_, thresh = cv2.threshold(gray, 0, 2255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
dst = (dst / (dst.max() - dst.min())*255).astype(np.uint8)
cv2.imshow('dst', dst)

localMx = cv2.dilate(dst, np.ones((50,50), np.uint8))
lm = np.zeros((rows, cols), np.uint8)
lm[(localMx==dst) & (dst != 0)] = 255
cv2.imshow('localMx', lm)

seeds = np.where(lm == 255)
seed = np.stack( (seeds[1], seeds[0]), axis=-1)
fill_mask = np.zeros((rows+2, cols+2), np.uint8)
for x,y in seed:
    ret = cv2.floodFill(mean, fill_mask, (x,y), (255,255,255), (10,10,10), (10,10,10))

cv2.imshow('floodFill', mean)

gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0)

ret, thresh = cv2.threshold(gray, 0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dst = ( (dst / (dst.max() - dst.min()))* 255).astype(np.uint8)

ret, sure_fg = cv2.threshold(dst, 0.5*dst.max(), 255, 0)
cv2.imshow('sure_fg', sure_fg)

_, bg_th = cv2.threshold(dst, 0.3*dst.max(), 255, cv2.THRESH_BINARY_INV)
bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2, 5)
bg_dst = ( (bg_dst / (bg_dst.max() - bg_dst.min()))*255).astype(np.uint8)
ret, sure_bg = cv2.threshold(bg_dst, 0.3*bg_dst.max(), 255, cv2.THRESH_BINARY)
cv2.imshow('sure_bg', sure_bg)

ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255, cv2.THRESH_BINARY_INV)
unkown = cv2.subtract(inv_sure_bg, sure_fg)
cv2.imshow('unkown', unkown)

_, markers = cv2.connectedComponents(sure_fg)

markers = markers+1
markers[unkown == 255] = 0
print('워터세드 전:', np.unique(markers))
colors = []
marker_show = np.zeros_like(img)
for mid in np.unique(markers):
    color = [int(j) for j in np.random.randint(0,255,3)]
    colors.append((mid, color))
    marker_show[markers==mid] = color
    coords = np.where(markers == mid)
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
cv2.imshow('before', marker_show)

markers = cv2.watershed(img, markers)
print('워터셰드 후:', np.unique(markers))

for mid, color in colors:
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    if coords[0].size <= 0:
        continue
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
marker_show[markers==-1] = (0,255,0)
cv2.imshow('watershed marker', marker_show)

img[markers==-1] = (0,255,0)
cv2.imshow('watershed', img)

mask = np.zeros((rows, cols), np.uint8)
mask[markers!=1] = 255
nobg = cv2.bitwise_and(img, img, mask=mask)
coin_label = [l for l in np.unique(markers) if (l != 1 and l!= -1)]

for i, label in enumerate(coin_label):
    mask[:,:] = 0
    mask[markers ==label] = 255
    coins = cv2.bitwise_and(img, img, mask=mask)
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x,y,w,h = cv2.boundingRect(contour[0])
    coin = coins[y:y+h, x:x+w]
    cv2.imshow('coin%d'%(i+1), coin)
    cv2.imwrite('./img/coin_test/coin%d.jpg'%(i+1), coin)
cv2.waitKey()
cv2.destroyAllWindows

워터세드 전: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
워터셰드 후: [-1  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


<function destroyAllWindows>